# Merging Dataframes #
When we merge dataframes, we munge together columns from one dataframe with columns from another dataframe using a property that the rows have in common. A new dataframe is created from the merge.

A merge is similar to a join in a relational database, where two or more tables share a key that provides a unique identifier for each row in the table. You join tables by selecting the rows in each table with common key values and return the columns from each table that you care about.

To merge dataframes, you use the PANDAS *merge* command, which takes the names of the two dataframes to merge and the type of merge to perform. There is also a join command that operates on the index of each dataframe. For a complete explanation of the difference between merge and join, check out this article: <a href="http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging" target="_blank">Merging and joining in Pandas</a>

#### Merge options ####
There are four options for merging dataframes:
* **Inner join** - return only the rows in both tables that share a key value
* **Left join** - return all rows from the left table and the rows in the right table that match key values in left table
* **Right join** - return all rows from right table and the rows in the left table that match key values in the right table
* **Outer join** - return all rows from both tables

### Inner join example ### 
Who was the manager for the St. Louis Cardinals in 2010?
#### Answer ####
* Managers.csv contains the list of all managers for all teams. 
* The manager for a team is listed by playerID. 
* Master.csv that lists the personal information for all players using the playerID as a unique identifier. 
* First, find the playerID from Managers. Then, merge it with nameFirst and nameLast columns in Master for that playerID.

In [1]:
import pandas as pd
from pybaseball.lahman import *

#This will get you the St. Louis manager in 2010
man = managers()
manSTL2010 = man.loc[(man['yearID']==2010)&(man['teamID']=='SLN')]
print(manSTL2010)
playerData = master()
#you can do a merge with Master to get the first and last name
result = pd.merge(manSTL2010, playerData, on='playerID', how='inner')
print(result)
#result has way more information than we need. 
#We can select only the columns we want to see.
result = pd.merge(manSTL2010[['yearID', 'teamID', 'playerID']], 
                 playerData[['playerID', 'nameFirst', 'nameLast']],
                 on='playerID', how='inner')
print(result)

       playerID  yearID teamID lgID  inseason    G   W   L  rank plyrMgr
3236  larusto01    2010    SLN   NL         1  162  86  76   2.0       N


## Left join example ##
Generate a dataframe of all pitchers since 2015 with an ERA < 4.0 and the gameID for the pitchers who also started an allstar game. The pitching table includes many more columns than the all star table, we can specify which columns we want from both tables.

### Why is this a left join? ###
We want see all rows in the pitching data, which is the left table. We only want the rows in all star, which is the right table, that match a row in our left table.

In [4]:
import pandas as pd
from pybaseball.lahman import *

allStar = all_star_full()
allStar = allStar.loc[(allStar['yearID']>2015) & (allStar['startingPos']==1.0)]
allStar.head(20)

pitch = pitching()
#I selected some variables that we might care about related to pitching quality
goodPitch = pitch.loc[(pitch['yearID']>2015) & (pitch['ERA'] < 4.0),
            ['playerID', 'yearID', 'teamID', 'W', 'L', 'G', 'GS',
              'CG', 'SHO', 'IPouts', 'H', 'HR', 'BB', 'SO', 'ERA']]
goodPitch.head(20)
allStarPitchers = pd.merge(goodPitch, allStar[['playerID','yearID', 'gameID',
                                    'teamID']],
                           on=['playerID', 'yearID'],how='left')
allStarPitchers.head(100)

,playerID,yearID,teamID_x,W,L,G,GS,CG,SHO,IPouts,H,HR,BB,SO,ERA,gameID,teamID_y
0,abadfe01,2016,MIN,1,4,39,0,0,0,102,27,2,14,29,2.65,NaN,NaN
1,achteaj01,2016,LAA,1,0,27,0,0,0,113,43,7,12,14,3.11,NaN,NaN
2,alexasc01,2016,KCA,0,0,17,0,0,0,57,24,1,7,16,3.32,NaN,NaN
3,allenco01,2016,CLE,3,5,67,0,0,0,204,41,8,27,87,2.51,NaN,NaN
4,altavda01,2016,SEA,0,0,15,0,0,0,37,11,0,1,10,0.73,NaN,NaN
5,alvarda01,2016,ATL,3,1,16,0,0,0,45,11,3,5,28,3.00,NaN,NaN
6,alvarjo02,2016,LAA,1,3,64,0,0,0,172,71,4,15,51,3.45,NaN,NaN
7,amarial01,2016,SDN,0,0,1,0,0,0,1,0,0,0,0,0.00,NaN,NaN
8,anderty01,2016,COL,5,6,19,19,0,0,343,119,12,28,99,3.54,NaN,NaN
9,aquinja01,2016,BAL,0,0,3,0,0,0,7,1,0,0,3,0.00,NaN,NaN


# Questions #
In groups of 2-3 or alone, find the answer to the following question in the Lahman data. Write your code in a code cell and submit your notebook to Canvas.

1. Who played second base for the Baltimore Orioles in 2016? The teamID for the Orioles is BAL. You need the first and last name, as well as the playerID?
2. If you reversed the order of the dataframes in the left-join example, and make all star the left dataframe and pitching the right dataframe, what would the merge return? Provide an explanation rather than the code.